In [36]:
import pandas as pd
import numpy as np
import yfinance as yf
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from tensorflow.keras.regularizers import l2
import tensorflow as tf



In [10]:
start_date = "2022-01-03"
end_date   = "2024-11-01"

data = yf.download("AAPL", start=start_date, end=end_date)
data.columns = data.columns.get_level_values(0) # from multi index to single index
print(data.head())
data_copy = data.copy()

[*********************100%***********************]  1 of 1 completed

Price                       Adj Close       Close        High         Low  \
Date                                                                        
2022-01-03 00:00:00+00:00  179.273605  182.009995  182.880005  177.710007   
2022-01-04 00:00:00+00:00  176.998337  179.699997  182.940002  179.119995   
2022-01-05 00:00:00+00:00  172.290222  174.919998  180.169998  174.639999   
2022-01-06 00:00:00+00:00  169.414093  172.000000  175.300003  171.639999   
2022-01-07 00:00:00+00:00  169.581558  172.169998  174.139999  171.029999   

Price                            Open     Volume  
Date                                              
2022-01-03 00:00:00+00:00  177.830002  104487900  
2022-01-04 00:00:00+00:00  182.630005   99310400  
2022-01-05 00:00:00+00:00  179.610001   94537600  
2022-01-06 00:00:00+00:00  172.699997   96904000  
2022-01-07 00:00:00+00:00  172.889999   86709100  


In [11]:
# data['days_range'] = data['High'] - data['Low']   # calculating the range of the day
# data['yesterdays_close'] = data['Adj Close'].shift(1)
# data['jump_from_yesterday'] = data['Open']- data['yesterdays_close']
# data['days_movement'] = data['Adj Close']-data['Open']
# data = data.drop(columns=['yesterdays_close','Close','High','Low','Open'])

# data_v1 = data.copy()
# del(data)

In [75]:
data = data_copy.copy()

data['target'] = data['Adj Close'].shift(-1)
data = data[data['target'].notnull()]
data = data.drop(columns=['Close','High','Low','Open'])
data = data.reset_index()
data['Date'] = data['Date'].dt.strftime('%Y-%m-%d')
data['Date'] = pd.to_datetime(data['Date']).dt.date
data.set_index('Date', inplace=True)



In [76]:
data.head()

Price,Adj Close,Volume,target
Date,,,
2022-01-03,179.273605,104487900,176.998337
2022-01-04,176.998337,99310400,172.290222
2022-01-05,172.290222,94537600,169.414093
2022-01-06,169.414093,96904000,169.581558
2022-01-07,169.581558,86709100,169.601242


In [77]:

# Assume start_date and end_date are defined, and data is your DataFrame
a = pd.date_range(start=start_date, end=end_date, freq="D")  # continuous dates
b = data.index  # our time series
diff_dates = a.difference(b)  # finds what in 'a' is not in 'b'

# Ensure diff_dates remains as Timestamps for compatibility with DataFrame index
# (no need to convert to string and back to datetime)
diff_dates = pd.to_datetime(diff_dates).date

# Ensure `td` is defined correctly
td = pd.Timedelta(days=1)  # Adjust according to your needs

for date in diff_dates:
    prev_date = date - td  # Previous date
    # Check if the previous date exists in the index
    if prev_date in data.index:  # prev_date is still a Timestamp
        prev_val = data.loc[prev_date]  # Access using loc
        data.loc[date] = prev_val  # Impute previous value
    else:
        print(f"Previous date {prev_date} not found in index.")  # Debug message or handling

data.sort_index(inplace=True)  # Sort the index
data.freq = "D"  # Set the time index frequency as daily


In [100]:
# Define split date and convert to date-only format to match the index
val_split_date = pd.to_datetime('2023-12-31').date()
test_split_date = pd.to_datetime('2024-06-30').date()

# Split the data
train = data[:val_split_date]  # Data up to and including 2023-12-31
val = data[val_split_date:test_split_date]   # Data from 2023-12-31 onward
test = data[test_split_date:]   # Data from 2023-12-31 onward




In [101]:

scaler_train = MinMaxScaler()
values = scaler_train.fit_transform(train[['Volume', 'Adj Close']])


# Define the window size
WINDOW = 30  # Window size of 14 days


train_data = tf.data.Dataset.from_tensor_slices(values) # Create a TensorFlow Dataset from the array
train_data = train_data.window(WINDOW + 1, shift=1, drop_remainder=True) # Create windowed dataset with the specified window size
train_data = train_data.flat_map(lambda x: x.batch(WINDOW + 1)) # Flatten the windowed dataset by batching

# Create features and target tuple
train_data = train_data.map(lambda x: (x[:-1], x[-1, 1]))  # x[:-1] for features, x[-1, 1] for target 'Adj Close' # Here, we use all columns for features, but only the 'Adj Close' (index 1) as the target
train_data = train_data.batch(32).prefetch(1) # Create batches of windows



scaler_val = MinMaxScaler()
val_values = scaler_val.fit_transform(val[['Volume', 'Adj Close']])

# Convert to TensorFlow Datasets similarly as before
#val_values = val[['Volume', 'Adj Close']].values
val_data = tf.data.Dataset.from_tensor_slices(val_values)
val_data = val_data.window(WINDOW + 1, shift=1, drop_remainder=True).flat_map(lambda x: x.batch(WINDOW + 1))
val_data = val_data.map(lambda x: (x[:-1], x[-1, 1])).batch(32).prefetch(1)


In [102]:

# Custom callback
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('mae') < 0.1:
            print("MAE under 0.1... Stopping training")
            self.model.stop_training = True

my_callback = CustomCallback()

# Learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 2:
        return 0.01
    else:
        return lr * 0.99

lr_scheduler = LearningRateScheduler(scheduler)

# LSTM model definition
lstm_model = Sequential([
    Bidirectional(LSTM(128, return_sequences=True), input_shape=[WINDOW, 2]),
    Dropout(0.2),
    #BatchNormalization(),
    Bidirectional(LSTM(128)),
    Dropout(0.2),
    #BatchNormalization(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.2),  # Experiment with different dropout rates
    Dense(1)
])

# Compile model
lstm_model.compile(
    loss=Huber(),
    optimizer=Adam(),
    metrics=['mae']
)

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Model summary
lstm_model.summary()


/opt/anaconda3/envs/Conda_3_12_7/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_20                │ (None, 30, 256)        │       134,144 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 30, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_21                │ (None, 256)            │       394,240 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 594,433 (2.27 MB)

 Trainable params: 594,433 (2.27 MB)

 Non-trainable params: 0 (0.00 B)

In [103]:
# lstm_history = lstm_model.fit(
#     train_data,
#     epochs=100,
#     callbacks=[lr_scheduler, my_callback]
# )

# Fit the model with training data
lstm_history = lstm_model.fit(
    train_data,
    epochs=100,
    validation_data=val_data,  # Ensure you define val_data appropriately
    callbacks=[lr_scheduler, my_callback, early_stopping, reduce_lr],
    batch_size=32  # Optional: specify batch size if it's not handled in your Dataset
)


Epoch 1/100
     22/Unknown 2s 34ms/step - loss: 1.6200 - mae: 0.4798

/opt/anaconda3/envs/Conda_3_12_7/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - loss: 1.5935 - mae: 0.4739 - val_loss: 0.3779 - val_mae: 0.3386 - learning_rate: 0.0100
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.2323 - mae: 0.1803 - val_loss: 0.0834 - val_mae: 0.2346 - learning_rate: 0.0100
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0496 - mae: 0.1476 - val_loss: 0.0798 - val_mae: 0.3357 - learning_rate: 0.0099
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0262 - mae: 0.1527 - val_loss: 0.1535 - val_mae: 0.4820 - learning_rate: 0.0098
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.0383 - mae: 0.1993 - val_loss: 0.3466 - val_mae: 0.7707 - learning_rate: 0.0097
Epoch 6/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.0688 - mae: 0.2545 - val_loss: 0.0742 - val_mae: 0.3097 - learning_rate: 0.0096
Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - loss: 0.0335 - mae: 0.1499 - val_loss: 0.4313 - val_mae: 0.8748 - learning_rate: 0.0095
Epoch 8/100
22/22 

2024-11-03 22:35:01.660063: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - loss: 0.0225 - mae: 0.1697 - val_loss: 0.0466 - val_mae: 0.2615 - learning_rate: 0.0040


In [104]:
# Assuming lstm_model is your trained model
model_save_path = '/Users/monilshah/Documents/02_NWU/09_MSDS_458_DL/99_group_project/StockPricePrediction/01_Bidirectional_LSTM/01_Models/bidirecttional_lstm.keras'  # Specify your desired path here

# Save the model
lstm_model.save(model_save_path)

print(f'Model saved to {model_save_path}')


Model saved to /Users/monilshah/Documents/02_NWU/09_MSDS_458_DL/99_group_project/StockPricePrediction/01_Bidirectional_LSTM/01_Models/bidirecttional_lstm.keras


## Test

In [105]:
# Assuming 'test' is your test DataFrame and scaler_train is your fitted MinMaxScaler

# Scale the test data
test_values = scaler_val.transform(test[['Volume', 'Adj Close']])  # Use only the features for scaling

# Define the window size
WINDOW = 30  # Same window size as used for training"

# Create a TensorFlow Dataset from the scaled test data
test_data = tf.data.Dataset.from_tensor_slices(test_values)

# Create windowed dataset with the specified window size
test_data = test_data.window(WINDOW + 1, shift=1, drop_remainder=True)

# Flatten the windowed dataset by batching
test_data = test_data.flat_map(lambda x: x.batch(WINDOW + 1))

# Create features and target tuple (for evaluation, you can use the last value as target)
test_data = test_data.map(lambda x: (x[:-1], x[-1, 1]))  # Here, x[-1, 1] corresponds to the 'Adj Close' price

# Create batches of windows
test_data = test_data.batch(32).prefetch(1)  # Adjust batch size as necessary


In [106]:
predictions = lstm_model.predict(test_data)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step


/opt/anaconda3/envs/Conda_3_12_7/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


In [107]:
predictions

array([[0.5081593 ],
       [0.5081597 ],
       [0.5081599 ],
       [0.5081601 ],
       [0.50816005],
       [0.5081599 ],
       [0.5081599 ],
       [0.5081599 ],
       [0.50815994],
       [0.50816005],
       [0.50816   ],
       [0.5081599 ],
       [0.5081601 ],
       [0.5081601 ],
       [0.5081601 ],
       [0.50816005],
       [0.50816023],
       [0.50815994],
       [0.5081598 ],
       [0.50816005],
       [0.50816005],
       [0.50816005],
       [0.50816005],
       [0.5081601 ],
       [0.50815976],
       [0.5081599 ],
       [0.50816   ],
       [0.50816   ],
       [0.50816005],
       [0.50816005],
       [0.50815994],
       [0.5081598 ],
       [0.50815946],
       [0.508159  ],
       [0.508159  ],
       [0.5081589 ],
       [0.50815874],
       [0.50815934],
       [0.5081595 ],
       [0.5081598 ],
       [0.50815994],
       [0.50815994],
       [0.50816   ],
       [0.50816005],
       [0.50816005],
       [0.50816005],
       [0.5081601 ],
       [0.508

In [108]:
# Reshape predictions to match the expected input shape of the scaler
predictions_reshaped = np.zeros((predictions.shape[0], 2))  # Create an array for two features
predictions_reshaped[:, 0] = predictions.flatten()  # Place the predictions in the first column

# Inverse scale the predictions
predictions_original = scaler_val.inverse_transform(predictions_reshaped)

# Extract the first column for actual sales predictions
predictions_original = predictions_original[:, 0]

# Output the original predictions
print(predictions_original)

[1.42262864e+08 1.42262953e+08 1.42262991e+08 1.42263041e+08
 1.42263028e+08 1.42262991e+08 1.42262991e+08 1.42262991e+08
 1.42263003e+08 1.42263028e+08 1.42263016e+08 1.42262991e+08
 1.42263041e+08 1.42263041e+08 1.42263041e+08 1.42263028e+08
 1.42263066e+08 1.42263003e+08 1.42262978e+08 1.42263028e+08
 1.42263028e+08 1.42263028e+08 1.42263028e+08 1.42263041e+08
 1.42262965e+08 1.42262991e+08 1.42263016e+08 1.42263016e+08
 1.42263028e+08 1.42263028e+08 1.42263003e+08 1.42262978e+08
 1.42262902e+08 1.42262801e+08 1.42262801e+08 1.42262789e+08
 1.42262751e+08 1.42262877e+08 1.42262915e+08 1.42262978e+08
 1.42263003e+08 1.42263003e+08 1.42263016e+08 1.42263028e+08
 1.42263028e+08 1.42263028e+08 1.42263041e+08 1.42263054e+08
 1.42263054e+08 1.42263054e+08 1.42263066e+08 1.42263092e+08
 1.42263079e+08 1.42263041e+08 1.42263066e+08 1.42263066e+08
 1.42263079e+08 1.42263104e+08 1.42263079e+08 1.42263066e+08
 1.42263041e+08 1.42263028e+08 1.42263028e+08 1.42263028e+08
 1.42263028e+08 1.422630

In [109]:
y_test = test['Volume']
y_test

Date
2024-06-30    82542700.0
2024-07-01    60402900.0
2024-07-02    58046200.0
2024-07-03    37369800.0
2024-07-04    37369800.0
                 ...    
2024-10-28    36087100.0
2024-10-29    35417200.0
2024-10-30    47070900.0
2024-10-31    47070900.0
2024-11-01    47070900.0
Name: Volume, Length: 125, dtype: float64